In [1]:
import pandas as pd
import numpy as np
from functions import *
import random
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
#Load models
#REMINDER: We are going to need to use 'eval' to get the models usable
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
print(len(models))


###SELECT MODEL TO OPTIMIZE
model_num = 4

5


In [3]:
models

[['model_0', 'model_1', 'model_2', 'model_3', 'model_4', 'model_06_13_20'],
 ["LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,\n                   intercept_scaling=1, l1_ratio=None, max_iter=50000,\n                   multi_class='auto', n_jobs=None, penalty='l2',\n                   random_state=75, solver='lbfgs', tol=5.993308915320019,\n                   verbose=0, warm_start=False)",
  "DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',\n                       max_depth=None, max_features=None, max_leaf_nodes=263,\n                       min_impurity_decrease=0.0, min_impurity_split=None,\n                       min_samples_leaf=1, min_samples_split=2,\n                       min_weight_fraction_leaf=0.0, presort='deprecated',\n                       random_state=75, splitter='best')",
  "RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,\n                       criterion='gini', max_depth=None, max_featur

In [4]:
df = pd.read_csv("../data/ufc-master.csv")

In [5]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
df = df[(df['Winner'] == 'Blue') | (df['Winner'] == 'Red')]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [6]:
#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

In [7]:
#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

In [8]:
#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[200:]
odds_train = odds_df[200:]
label_train = label_df[200:]

df_test = df[:200]
odds_test = odds_df[:200]
label_test = label_df[:200]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

200
200
200
4033
4033
4033


In [9]:
display(df_train)
display(df_test)

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,sub_dif,height_dif,reach_dif,age_dif,sig_str_dif,avg_sub_att_dif,avg_td_dif,empty_arena,constant_1,label
201,Darko Stosic,Kennedy Nzechukwu,-120.0,100.0,83.333333,100.000000,2019-08-03,"Newark, New Jersey, USA",USA,Blue,...,0.0,12.70,17.78,0.0,1.960000,0.000000,-3.420000,0.0,1.0,1
202,Mickey Gall,Salim Touahri,100.0,-120.0,100.000000,83.333333,2019-08-03,"Newark, New Jersey, USA",USA,Red,...,-4.0,-10.16,-5.08,-2.0,0.380000,-2.700000,-1.500000,0.0,1.0,0
203,Antonina Shevchenko,Lucie Pudilova,-160.0,140.0,62.500000,140.000000,2019-08-03,"Newark, New Jersey, USA",USA,Red,...,0.0,0.00,0.00,9.0,-0.750000,-0.400000,-0.510000,0.0,1.0,0
204,Jordan Espinosa,Matt Schnell,-155.0,145.0,64.516129,145.000000,2019-08-03,"Newark, New Jersey, USA",USA,Blue,...,1.0,5.08,2.54,0.0,-0.520000,1.200000,-0.210000,0.0,1.0,1
205,Lauren Murphy,Mara Romero Borella,180.0,-220.0,180.000000,45.454545,2019-08-03,"Newark, New Jersey, USA",USA,Red,...,1.0,2.54,5.08,3.0,-0.780000,0.300000,0.650000,0.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4301,Duane Ludwig,Darren Elkins,-155.0,135.0,64.516129,135.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Blue,...,0.0,0.00,2.54,6.0,-13.666667,0.000000,0.000000,0.0,1.0,1
4302,John Howard,Daniel Roberts,-210.0,175.0,47.619048,175.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,7.62,7.62,-2.0,-18.000000,-1.000000,-4.666667,0.0,1.0,0
4303,Brendan Schaub,Chase Gormley,-260.0,220.0,38.461538,220.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,-2.54,-2.12,0.0,-4.000000,1.000000,1.000000,0.0,1.0,0
4304,Mike Pierce,Julio Paulino,-420.0,335.0,23.809524,335.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,10.16,7.62,-5.0,-40.500000,0.000000,-3.500000,0.0,1.0,0


,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,sub_dif,height_dif,reach_dif,age_dif,sig_str_dif,avg_sub_att_dif,avg_td_dif,empty_arena,constant_1,label
0,Jessica Eye,Cynthia Calvillo,120.0,-130.0,120.000000,76.923077,2020-06-13,"Las Vegas, Nevada, USA",USA,Blue,...,3.0,-5.08,-5.08,-1.0,0.33,0.9,1.37,1.0,1.0,1
1,Karl Roberson,Marvin Vettori,210.0,-230.0,210.000000,43.478261,2020-06-13,"Las Vegas, Nevada, USA",USA,Blue,...,-1.0,-2.54,0.00,-3.0,0.89,-0.2,0.36,1.0,1.0,1
2,Charles Rosa,Kevin Aguilar,170.0,-185.0,170.000000,54.054054,2020-06-13,"Las Vegas, Nevada, USA",USA,Red,...,-2.0,-5.08,10.16,-2.0,0.52,-1.0,-2.11,1.0,1.0,0
3,Andre Fili,Charles Jourdain,-220.0,200.0,45.454545,200.000000,2020-06-13,"Las Vegas, Nevada, USA",USA,Red,...,0.0,-5.08,-12.70,-5.0,0.85,-0.2,-2.49,1.0,1.0,0
4,Jordan Espinosa,Mark De La Rosa,-167.0,157.0,59.880240,157.000000,2020-06-13,"Las Vegas, Nevada, USA",USA,Red,...,1.0,0.00,-10.16,-5.0,-0.97,-0.2,-0.19,1.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,Colby Covington,Robbie Lawler,-300.0,250.0,33.333333,250.000000,2019-08-03,"Newark, New Jersey, USA",USA,Red,...,-2.0,0.00,5.08,-6.0,-0.60,-0.2,-4.20,0.0,1.0,0
197,Jim Miller,Clay Guida,-140.0,130.0,71.428571,130.000000,2019-08-03,"Newark, New Jersey, USA",USA,Red,...,-8.0,-2.54,-2.54,-2.0,-0.33,-1.3,1.62,0.0,1.0,0
198,Joaquim Silva,Nasrat Haqparast,315.0,-380.0,315.000000,26.315789,2019-08-03,"Newark, New Jersey, USA",USA,Blue,...,0.0,5.08,7.62,7.0,1.33,-0.7,0.05,0.0,1.0,1
199,Trevin Giles,Gerald Meerschaert,-170.0,150.0,58.823529,150.000000,2019-08-03,"Newark, New Jersey, USA",USA,Blue,...,3.0,2.54,7.62,-4.0,-0.88,1.6,1.09,0.0,1.0,1


In [10]:
#1. Set features
#2. Set Hyperparameters
#3. Set EV
#4. Remove Features

In [11]:
models[1][model_num]

'GaussianNB(priors=None, var_smoothing=1e-12)'

In [12]:
test_model_name = models[0][model_num]
test_model = eval(models[1][model_num])
test_model_features = eval(models[2][model_num])
test_model_ev = eval(models[3][model_num])

In [13]:
old_test_model = test_model
old_test_model_features = test_model_features
old_test_model_ev = test_model_ev

In [14]:
#1. set features
my_pos_features = ['R_fighter', 'B_fighter', 'R_odds', 'B_odds', 'R_ev', 'B_ev',
       'location', 'country', 'title_bout', 'weight_class', 'gender',
       'no_of_rounds', 'B_current_lose_streak', 'B_current_win_streak',
       'B_draw', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT',
       'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses',
       'B_total_rounds_fought', 'B_total_title_bouts',
       'B_win_by_Decision_Majority', 'B_win_by_Decision_Split',
       'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission',
       'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Stance', 'B_Height_cms',
       'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak',
       'R_current_win_streak', 'R_draw', 'R_avg_SIG_STR_landed',
       'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct',
       'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought',
       'R_total_title_bouts', 'R_win_by_Decision_Majority',
       'R_win_by_Decision_Split', 'R_win_by_Decision_Unanimous',
       'R_win_by_KO/TKO', 'R_win_by_Submission',
       'R_win_by_TKO_Doctor_Stoppage', 'R_wins', 'R_Stance', 'R_Height_cms',
       'R_Reach_cms', 'R_Weight_lbs', 'R_age', 'B_age', 'lose_streak_dif',
       'win_streak_dif', 'longest_win_streak_dif', 'win_dif', 'loss_dif',
       'total_round_dif', 'total_title_bout_dif', 'ko_dif', 'sub_dif',
       'height_dif', 'reach_dif', 'age_dif', 'sig_str_dif', 'avg_sub_att_dif',
       'avg_td_dif', 'empty_arena']

In [15]:
print(test_model_name)
print(test_model)
print(test_model_features)
print(test_model_ev)



model_4
GaussianNB(priors=None, var_smoothing=1e-12)
['height_dif', 'sub_dif', 'B_draw', 'avg_td_dif', 'R_win_by_Decision_Split', 'age_dif', 'B_current_win_streak', 'R_odds']
0


In [16]:
keep_going = True
while(keep_going):
    #1. Set Features
    #get_best_features(pos_features, m, df, cur_features, labels, odds, scale=False)
    test_model_features = (get_best_features(my_pos_features, test_model, df_train, test_model_features, label_train, odds_train, 
                                             min_ev=test_model_ev))

    #2 Set hyperparameters
    #def tune_hyperparameters(input_model, input_features, input_df, input_labels, odds_input):
    test_model = tune_hyperparameters(test_model, test_model_features, df_train, label_train, odds_train, 
                                      min_ev=test_model_ev)

    #3. Set EV
    #def tune_ev(input_model, input_features, input_df, input_labels, odds_input, verbose=False):
    test_model_ev = tune_ev(test_model, test_model_features, df_train, label_train, odds_train, verbose=False)

    old_test_model_features = test_model_features #This prevents
                                                  #an uneccesary loop
    
    
    #4. Remove Features
    #def remove_to_improve(cur_features, m, df, labels, odds, scale=False, min_ev = 0):
    test_model_features = remove_to_improve(test_model_features, test_model, df_train, label_train, odds_train, min_ev = test_model_ev)    
    keep_going = False
    if old_test_model != test_model:
        print("The hyperparameters are different")
        print("OLD:")
        print(old_test_model)
        print("NEW:")
        print(test_model)
        keep_going = True
        old_test_model = test_model
    if old_test_model_features != test_model_features:
        print("The features are different")
        print("OLD:")
        print(old_test_model_features)
        print("NEW:")
        print(test_model_features)
        keep_going = True
        old_test_model_features = test_model_features
    if old_test_model_ev != test_model_ev:
        print("The EV is different")
        print("OLD:")
        print(old_test_model_ev)
        print("NEW:")
        print(test_model_ev)
        keep_going = True
        old_test_model_ev = test_model_ev
        
    

Current best score is: 10243.684894935768
Feature: B_avg_SUB_ATT Score: 12850.582433122721
Feature: R_win_by_TKO_Doctor_Stoppage Score: 13385.883012208295
The best feature was R_win_by_TKO_Doctor_Stoppage.  It scored 13385.883012208295
Current best score is: 13385.883012208295
Feature: B_current_lose_streak Score: 13448.018893927869
Feature: B_avg_SIG_STR_pct Score: 14665.409944277479
Feature: B_avg_SUB_ATT Score: 18203.220963192598
The best feature was B_avg_SUB_ATT.  It scored 18203.220963192598
Current best score is: 18203.220963192598
Feature: gender Score: 18556.4704019333
Feature: B_current_lose_streak Score: 20271.82136881636
Feature: B_Height_cms Score: 21272.742458381876
The best feature was B_Height_cms.  It scored 21272.742458381876
Current best score is: 21272.742458381876
Feature: sig_str_dif Score: 22636.701588762713
The best feature was sig_str_dif.  It scored 22636.701588762713
Current best score is: 22636.701588762713
Feature: B_win_by_Submission Score: 22803.618515135

var_smoothing: 1e-07 Score: 19933.510982982658
var_smoothing: 1e-06 Score: 14570.620414487115
0 24170.514701678803
1 23776.491588498437
2 22825.349591969105
3 20981.49421062113
4 19452.354464561973
5 19361.78231226253
6 19221.470417892095
7 18616.316576777375
8 18812.06814173564
9 17892.95786720109
10 16122.73051981922
11 14885.516790500691
12 14335.021439923657
13 13414.807562452992
14 12641.719162227202
15 11361.703638316558
16 11489.763228852345
17 10968.343367000547
18 11616.380907699871
19 11652.589148459627
20 10487.942815281182
21 9261.620414375553
22 9924.469902332567
23 9364.916007324415
24 9026.501950671363
25 8536.068712318274
26 9122.394872027327
27 8794.866552190213
28 7697.62890933031
29 8417.556718522384
30 8910.224848763604
31 10071.546373438772
32 10756.74709555718
33 9952.823357731359
34 9767.4866880337
35 10389.204074030247
36 10237.685867570792
37 10011.147963467098
38 10195.292139118414
39 9605.451037247236
40 9315.240800574153
41 9994.174191885824
42 9398.00803940

In [17]:
#Get new score:
#def evaluate_model(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels, test_odds, verbose=True):
score = evaluate_model(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)

(3201, 14)

(3201,)

(3201, 2)

(200, 14)

(200,)

(200, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,120.0,-130.0,4.794698e-18,1.000000,1
1,210.0,-230.0,5.689068e-18,1.000000,1
2,170.0,-185.0,6.657905e-01,0.334209,0
3,-220.0,200.0,8.269296e-01,0.173070,0
4,-167.0,157.0,8.239596e-01,0.176040,0
...,...,...,...,...,...
195,-300.0,250.0,8.918896e-01,0.108110,0
196,-140.0,130.0,5.030155e-01,0.496984,0
197,315.0,-380.0,1.537493e-01,0.846251,1
198,-170.0,150.0,8.259885e-01,0.174012,1



          Number of matches: 200
          Number of bets: 191
          Number of winning bets: 125
          Number of losing bets: 66
          Number of underdog bets: 65
          Number of underdog wins: 30
          Number of underdog losses: 35
          Number of Favorite bets: 120
          Number of favorite wins: 90
          Number of favorite losses: 30
          Number of even bets: 6
          Number of even wins: 5
          Number of even losses: 1
          Profit: 2324.902530428353
          Profit per bet: 12.17226455721651
          Profit per match: 11.624512652141766
          
          


print(test_model_name) 
print(models[0][model_num])
print()
print(test_model)
print(eval(models[1][model_num]))
print()
print(test_model_features) 
print(eval(models[2][model_num]))
print()
print(test_model_ev)
print(eval(models[3][model_num]))

In [18]:
models[0][model_num] =  test_model_name 
models[1][model_num] = test_model
models[2][model_num] = test_model_features
models[3][model_num] = test_model_ev
models[4][model_num] = score

In [19]:
with open('models.csv', 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    for row in models:
        print("HI")
        writer.writerow(row)
    
outfile.close()

HI
HI
HI
HI
HI


In [20]:
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
    
print(len(data))

5


In [21]:
data

[['model_0', 'model_1', 'model_2', 'model_3', 'model_4', 'model_06_13_20'],
 ["LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,\n                   intercept_scaling=1, l1_ratio=None, max_iter=50000,\n                   multi_class='auto', n_jobs=None, penalty='l2',\n                   random_state=75, solver='lbfgs', tol=5.993308915320019,\n                   verbose=0, warm_start=False)",
  "DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',\n                       max_depth=None, max_features=None, max_leaf_nodes=263,\n                       min_impurity_decrease=0.0, min_impurity_split=None,\n                       min_samples_leaf=1, min_samples_split=2,\n                       min_weight_fraction_leaf=0.0, presort='deprecated',\n                       random_state=75, splitter='best')",
  "RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,\n                       criterion='gini', max_depth=None, max_featur